In [3]:
# The problem:
# Here in my country we have two day of holidays more the days of the weekend, so we have 5 day of holidays.
#   Everybody will go out of the city.
#
# The problem is: ¿where i going to?
# I like two cities "Viña del mar" and "La ligua" and the selection criteria will be restaurantes,
#   the number of restaurant in each city, so:
#
#    1.- Search for a hostel closer to the center of the city
#
#    2.- Search the number of restaurant 500 ft arraund


In [4]:

import pandas as pd
import numpy  as np

import requests                          # library to handle requests
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import folium                            # plotting library


### Credentials for Foursquare


In [5]:

CLIENT_ID = 'WDKVMU4XRDDSFDYUT2EZT5J1HEOY2GZJSED3DERKVXZVZQPB'
CLIENT_SECRET = '0JETOZ2PW3A21GPGVWIC02UKJM3JMOEUNQK0NAL0P4HSO04F'
VERSION = '20190825'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WDKVMU4XRDDSFDYUT2EZT5J1HEOY2GZJSED3DERKVXZVZQPB
CLIENT_SECRET:0JETOZ2PW3A21GPGVWIC02UKJM3JMOEUNQK0NAL0P4HSO04F




### Now we search the number of restaurants in Viña del Mar

#### 1.- We search a Hotel in Viña del Mar

In [116]:
address = 'Viña del Mar, Chile, CL'

geolocator = Nominatim(user_agent="foursquare_agent")
location   = geolocator.geocode(address)
latitude   = location.latitude
longitude  = location.longitude

# Directly from Google Maps
latitude   = -33.0152001
longitude  = -71.5535746

print(latitude, longitude)

-33.0152001 -71.5535746


In [117]:
search_query = 'Hotel'
radius = 500

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WDKVMU4XRDDSFDYUT2EZT5J1HEOY2GZJSED3DERKVXZVZQPB&client_secret=0JETOZ2PW3A21GPGVWIC02UKJM3JMOEUNQK0NAL0P4HSO04F&ll=-33.0152001,-71.5535746&v=20190825&query=Hotel&radius=500&limit=30'

In [125]:
results = requests.get(url).json()
#results

In [126]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# Tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()
#len(dataframe)

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",False,4d963bbc2bd6f04d1bcd3150,4 norte 646,CL,Viña del Mar,Chile,1 poniente,501,"[4 norte 646 (1 poniente), Viña del Mar, Valpa...","[{'label': 'display', 'lat': -33.0194478948408...",-33.019448,-71.551786,NaN,Valparaíso,Hotel Mare Nostrum,v-1567748183
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,5a6e326660d11b2a4c0ae95a,NaN,CL,Viña del Mar,Chile,NaN,197,"[2520000 Viña del Mar, Valparaíso, Chile]","[{'label': 'display', 'lat': -33.0148735064648...",-33.014874,-71.555657,2520000,Valparaíso,Hotel Atton San Martin,v-1567748183
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4c938ad972dd224bf5749191,Avda. San Martin 476,CL,Viña del Mar,Chile,NaN,383,"[Avda. San Martin 476, Viña del Mar, Valparaís...","[{'label': 'display', 'lat': -33.0167117939947...",-33.016712,-71.557261,NaN,Valparaíso,Hotel Ankara,v-1567748183
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4d6917ce4c4637045f2f9db3,NaN,CL,NaN,Chile,NaN,130,[Chile],"[{'label': 'display', 'lat': -33.0163576334832...",-33.016358,-71.553342,NaN,NaN,Hotel Vancouver,v-1567748183
4,"[{'id': '4bf58dd8d48988d1f8931735', 'name': 'B...",False,5247693911d2f7417c1df973,NaN,CL,NaN,Chile,NaN,156,[Chile],"[{'label': 'display', 'lat': -33.015354, 'lng'...",-33.015354,-71.555245,NaN,NaN,Apart Hotel Rent A Home,v-1567748183


In [127]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
df_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)

#print(dataframe_filtered)

# clean column names by keeping only last term
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]


df_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Hotel Mare Nostrum,Building,4 norte 646,CL,Viña del Mar,Chile,1 poniente,501,"[4 norte 646 (1 poniente), Viña del Mar, Valpa...","[{'label': 'display', 'lat': -33.0194478948408...",-33.019448,-71.551786,NaN,Valparaíso,4d963bbc2bd6f04d1bcd3150
1,Hotel Atton San Martin,Hotel,NaN,CL,Viña del Mar,Chile,NaN,197,"[2520000 Viña del Mar, Valparaíso, Chile]","[{'label': 'display', 'lat': -33.0148735064648...",-33.014874,-71.555657,2520000,Valparaíso,5a6e326660d11b2a4c0ae95a
2,Hotel Ankara,Hotel,Avda. San Martin 476,CL,Viña del Mar,Chile,NaN,383,"[Avda. San Martin 476, Viña del Mar, Valparaís...","[{'label': 'display', 'lat': -33.0167117939947...",-33.016712,-71.557261,NaN,Valparaíso,4c938ad972dd224bf5749191
3,Hotel Vancouver,Hotel,NaN,CL,NaN,Chile,NaN,130,[Chile],"[{'label': 'display', 'lat': -33.0163576334832...",-33.016358,-71.553342,NaN,NaN,4d6917ce4c4637045f2f9db3
4,Apart Hotel Rent A Home,Bed & Breakfast,NaN,CL,NaN,Chile,NaN,156,[Chile],"[{'label': 'display', 'lat': -33.015354, 'lng'...",-33.015354,-71.555245,NaN,NaN,5247693911d2f7417c1df973


In [48]:
#df_filtered.name

In [49]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
#folium.features.CircleMarker(
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='black',
    popup='Viña del Mar',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.name):
#    folium.features.CircleMarker(
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map



### We select the  'Apart Hotel Rent A Home'

#### 2.- Now we search the number of restaurant around of 'Hotel Terra'

In [53]:
# We take the latitude and longitude of 'Apart Hotel Rent A Home'

LatLng = df_filtered.loc[df_filtered['name'] == 'Apart Hotel Rent A Home'].loc[:, ['lat', 'lng'] ]
#id = ids.loc[:, ['lat', 'lng'] ]
LatLng

,lat,lng
4,-33.015354,-71.555245


In [77]:
# We search the number of Restaurant arraund of 'Apart Hotel Rent A Home' Hotel
latitude   = LatLng.lat.item()
longitude  = LatLng.lng.item()

search_query = 'Restaurant'
radius = 500

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)


In [64]:
# The result of query search
results = requests.get(url).json()
#results

In [67]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# Tranform venues into a dataframe
dataframe = json_normalize(venues)

print("The number of restaurant around of 'Apart Hotel Rent A Home' is:", len(dataframe))
#dataframe.shape

The number of restaurant around of 'Apart Hotel Rent A Home' is: 30





# Now we search the number of restaurants in Iquique

#### 1.- We search a Hotel in Iquique

In [68]:
# The coordinates of Iquique city

latitude   = -20.2185663
longitude  = -70.1499344

In [69]:
search_query = ''
search_query = 'Hotel'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

Hotel .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=WDKVMU4XRDDSFDYUT2EZT5J1HEOY2GZJSED3DERKVXZVZQPB&client_secret=0JETOZ2PW3A21GPGVWIC02UKJM3JMOEUNQK0NAL0P4HSO04F&ll=-20.2185663,-70.1499344&v=20190825&query=Hotel&radius=500&limit=30'

In [72]:
results = requests.get(url).json()
#results

In [73]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# Tranform venues into a dataframe
dataframe = json_normalize(venues)
#dataframe.head()
len(dataframe)
#dataframe.shape

30

In [74]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
df_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)

#print(dataframe_filtered)

# clean column names by keeping only last term
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]


df_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Hotel Tikoo,Hotel,Ramírez 1051,CL,Iquique,Chile,NaN,75,"[Ramírez 1051, 1101043 Iquique, Tarapacá, Chile]","[{'label': 'display', 'lat': -20.2179947192690...",-20.217995,-70.149546,1101043,Tarapacá,4f255738e4b047657a9a5714
1,Hotel Barros Arana,Hotel,Barros Arana 1302,CL,Iquique,Chile,NaN,356,"[Barros Arana 1302, Iquique, Tarapacá, Chile]","[{'label': 'display', 'lat': -20.2207702759607...",-20.220770,-70.147457,NaN,Tarapacá,4dd30b72e4cd1b1913103926
2,HOTEL ESMERALDA,Bed & Breakfast,OBISPO LABBE 1386,CL,Iquique,Chile,RIQUELME,299,"[OBISPO LABBE 1386 (RIQUELME), 1101330 Iquique...","[{'label': 'display', 'lat': -20.2212044641641...",-20.221204,-70.150502,1101330,Tarapacá,52fe2a34498efa6401c60fbb
3,Hotel Plaza,Hotel,NaN,CL,Iquique,Chile,NaN,263,"[Iquique, Región de Ta, Chile]","[{'label': 'display', 'lat': -20.2175511957617...",-20.217551,-70.152212,NaN,Región de Ta,4e5fa8b9d164ed2b48f192ba
4,Hotel Inti-llanka,Hotel Bar,Obispo Labbe,CL,Iquique,Chile,Almirante Latorre,307,"[Obispo Labbe (Almirante Latorre), Iquique, Ta...","[{'label': 'display', 'lat': -20.2158266926146...",-20.215827,-70.150273,NaN,Tarapacá,4bb3c5a514cfd13abc8616ab


In [75]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
#folium.features.CircleMarker(
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='black',
    popup='Iquique',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.name):
#    folium.features.CircleMarker(
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### The Hotel more closely to the beach is 'Hotel Terra'


#### 2.- Now we search the number of restaurant around of 'Hotel Terra'


In [107]:
# We take the latitude and longitude of 'Hotel Terra'

LatLng = df_filtered.loc[df_filtered['name'] == 'Hotel Casa Baquedano'].loc[28, ['lat', 'lng'] ]
#id = ids.loc[:, ['lat', 'lng'] ]
LatLng

lat   -20.2219
lng   -70.1519
Name: 28, dtype: object

In [110]:
# We search the number of Restaurant arraund of 'Hotel Casa Baquedano' Hotel
latitude   = LatLng.lat.item()
longitude  = LatLng.lng.item()

search_query = 'Restaurant'
radius = 500

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)


In [111]:
# The result of query search
results = requests.get(url).json()
#results

In [113]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# Tranform venues into a dataframe
dataframe = json_normalize(venues)

print("The number of restaurant around of 'Hotel Casa Baquedano' is:", len(dataframe))

The number of restaurant around of 'Hotel Casa Baquedano' is: 18


# Conclusion

The hotel in 'Viña del Mar' have 30 restaurant around. And the hotel in Iquique just have 18 restaurant around closely.
So, i will travel to the city of Viña del Mar
